In [1]:
import os
import sys
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import model_from_json
from sklearn.metrics import f1_score
import numpy as np
from PIL import Image
import math

from constant_values import *
from img_preprocessing import *
from LIDAR_model import *

Using TensorFlow backend.


In [2]:
data_dir = '../Data/training/'
img_dir = data_dir + 'images/'
gt_dir = data_dir + 'groundtruth/'

# Extract it into numpy arrays.
img = extract_data(img_dir, TRAINING_SIZE)
gt = extract_labels(gt_dir, TRAINING_SIZE)

Loading ../Data/training/images/satImage_001.png
Loading ../Data/training/images/satImage_002.png
Loading ../Data/training/images/satImage_003.png
Loading ../Data/training/images/satImage_004.png
Loading ../Data/training/images/satImage_005.png
Loading ../Data/training/images/satImage_006.png
Loading ../Data/training/images/satImage_007.png
Loading ../Data/training/images/satImage_008.png
Loading ../Data/training/images/satImage_009.png
Loading ../Data/training/images/satImage_010.png
Loading ../Data/training/images/satImage_011.png
Loading ../Data/training/images/satImage_012.png
Loading ../Data/training/images/satImage_013.png
Loading ../Data/training/images/satImage_014.png
Loading ../Data/training/images/satImage_015.png
Loading ../Data/training/images/satImage_016.png
Loading ../Data/training/images/satImage_017.png
Loading ../Data/training/images/satImage_018.png
Loading ../Data/training/images/satImage_019.png
Loading ../Data/training/images/satImage_020.png
Loading ../Data/trai

In [3]:
#Test model simple

# Create model
# https://keras.io/models/sequential/
#mod = Sequential()

# Add a layer
# https://keras.io/layers/about-keras-layers/
# https://keras.io/activations/
#mod.add(Conv2D(64, kernel_size=IMG_PATCH_SIZE, activation='relu',input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
#mod.add(Flatten())
#mod.add(Dense(2, activation='softmax'))
        
        
# https://keras.io/optimizers/
# https://keras.io/losses/
#mod.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy']) #f1 = 0.43
#mod.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) #f1 = 0.43

In [4]:
model = model_Lidar((16, 16, 3))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
TEST_SIZE = 0.33 # To be studied

# Step 0: Shuffle samples
np.random.seed(0)
np.random.shuffle(img)
np.random.seed(0)
np.random.shuffle(gt)

# Step 1: Split into validation and training set     
Xtrain, Xtest, gt_train, gt_test = train_test_split(img, gt, test_size=TEST_SIZE, random_state=1)

# Step 2: Give weights to classes ?
c_weight = {1: 3., 
            0: 1.}

# Step 3: Generate Generators
train_datagenerator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
test_datagenerator  = ImageDataGenerator()

train_generator = train_datagenerator.flow(Xtrain, gt_train, batch_size=BATCH_SIZE)
test_generator  = test_datagenerator.flow(Xtest, gt_test, batch_size=BATCH_SIZE)

# Step 4: Early stop
early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='max', restore_best_weights=True)

# Step 5: Training
model.fit_generator(train_generator,
            validation_data=test_generator,
            steps_per_epoch=math.ceil(TRAINING_SIZE / BATCH_SIZE), #len(train_generator)/16,
            epochs=NUM_EPOCHS,
            callbacks = [early_stop_callback],
            class_weight=c_weight,
            validation_steps=math.ceil(len(test_generator)/BATCH_SIZE))

Epoch 1/100
4/4 [==============================] - 22s 5s/step - loss: 1.0609 - accuracy: 0.3438 - val_loss: 0.7243 - val_accuracy: 0.2287
Epoch 2/100


/home/clelie/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


4/4 [==============================] - 19s 5s/step - loss: 1.0273 - accuracy: 0.6094 - val_loss: 0.6433 - val_accuracy: 0.7271
Epoch 3/100
4/4 [==============================] - 18s 5s/step - loss: 1.0548 - accuracy: 0.5156 - val_loss: 0.7021 - val_accuracy: 0.2287
Epoch 4/100
4/4 [==============================] - 18s 5s/step - loss: 1.0926 - accuracy: 0.2344 - val_loss: 0.7105 - val_accuracy: 0.2759
Epoch 5/100
4/4 [==============================] - 18s 5s/step - loss: 1.1264 - accuracy: 0.3750 - val_loss: 0.7120 - val_accuracy: 0.2668
Epoch 6/100
4/4 [==============================] - 19s 5s/step - loss: 1.0432 - accuracy: 0.2969 - val_loss: 0.7310 - val_accuracy: 0.2378
Epoch 7/100
4/4 [==============================] - 18s 4s/step - loss: 0.9275 - accuracy: 0.3594 - val_loss: 0.6618 - val_accuracy: 0.7576
Epoch 8/100
4/4 [==============================] - 18s 4s/step - loss: 1.0489 - accuracy: 0.7500 - val_loss: 0.6641 - val_accuracy: 0.7454
Epoch 9/100
4/4 [======================

In [ ]:
prediction_training_dir = "predictions_training_vgg_model/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)

for i in range(1, TRAINING_SIZE+1):
    pimg = get_prediction_with_groundtruth(train_data_filename, i, mod_vgg)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(train_data_filename, i, mod_vgg)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")


# serialize model to JSON
OUTPUT_FILENAME = "simple_model"
model_json = mod.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
mod.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

validation_data_prediction = mod.predict_classes(data_te)
validation_label = []
for e in label_te:
    if (e[0] == 0):
        validation_label.append(1)
    else:
        validation_label.append(0)
        
validation_label = np.array(validation_label)     
print("F1 score = "+str(f1_score(validation_data_prediction, validation_label, labels=['1'])))